# Multishot Prompting via learning from Historical Conversation
Learning from historical conversations (Which could be stored in databases) allows the model to cache information and utilize in particular conversation.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [ ]:
archievedConversation = [{"role": "user", "content": "Customer A: Hi, I am looking to buy a belt."},
                        {"role": "assistant", "content": "I am sorry but we do not sell belts in this store; but you can find them in our second store.\
                        Do you want me to tell you the address of that store?"}
                        ,{"role": "user", "content": "Customer A: Yes please tell me the location."},
                        {"role": "assistant", "content": "Please walk straight from this store and then take a right, the second store is 3 streets after next to a burger joint." }]

def chat(message, history):

    if 'belt' in message:
        messages = [{"role": "system", "content": system_message}] + archievedConversation + history + [{"role": "user", "content": message}]
    else:
        messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)